# VP Function Verification

Reads `vp_function_mapping.csv` and calls each VP function at the latest block to verify the mapping is correct.

In [1]:
import pandas as pd
from pathlib import Path
from multicall import Call

from mainnet_launch.data_fetching.get_state_by_block import (
    get_state_by_one_block,
    safe_normalize_with_bool_success,
    build_blocks_to_use, get_raw_state_by_blocks
)
from mainnet_launch.constants import ETH_CHAIN, BASE_CHAIN

mapping = pd.read_csv(
    Path(__file__).parent / "vp_function_mapping.csv" if "__file__" in dir() else "vp_function_mapping.csv"
)
mapping

/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/eth_abi/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


,autopool,destination_vault_address,destination_name,pool_address,exchange,vp_function,vp_status,notes
0,autoETH,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,autoETH (Idle),0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,tokemak,N/A - always 1.0,OK,autopool itself - VP is always 1.0
1,autoETH,0x1Ea622fa030e4a78F4CC2f305dd3c08DA3F08573,Curve.fi Factory Pool: ETHx-ETH,0x59Ab5a5b5d617E478a2479B0cAD80DA7e2831492,curve,get_virtual_price()(uint256),OK,VP ~1.036-1.045 reasonable for Curve
2,autoETH,0xC001f23397dB71B17602Ce7D90a983Edc38DB0d1,Curve.fi Factory Pool: ETHx-ETH (old),0x59Ab5a5b5d617E478a2479B0cAD80DA7e2831492,curve,get_virtual_price()(uint256),SUSPECT,VP ~1.0025 - same pool as 0x1Ea6 but VP way lo...
3,autoETH,0x3772973f8F399D74488D5cF3276C032E0afC8A6f,Curve.fi Factory Pool: OETH,0x94B17476A93b3262d87B9a326965D1E91f9c13E7,curve,get_virtual_price()(uint256),SUSPECT,VP ~1.0023 - hardcoded override in augment_pla...
4,autoETH,0x28109255F49f0A02BB31358B3e0b5cb7512EB4eC,OETH/WETH,0xcc7d5785AD5755B6164e21495E07aDb0Ff11C2A8,curve,get_virtual_price()(uint256),SUSPECT,VP ~1.0002 - only 2 events; very low VP sugges...
5,autoETH,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,Instadapp ETH v2,0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78,fluid,exchangePrice()(uint256),OK,VP ~1.17-1.20 reasonable; hardcoded override i...
6,autoETH,0xd100c932801390fdeBcE11F26f611D4898b44236,wstETH (Holding),0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0,lido,stEthPerToken()(uint256),OK,VP ~1.21 reasonable for wstETH; hardcoded over...
7,autoETH,0x3F55eedDe51504E6Ed0ec30E8289b4Da11EdB7F9,osETH/rETH (new),0xe080027Bd47353b5D1639772b4a75E9Ed3658A0d,curve,get_virtual_price()(uint256),OK,VP ~1.028-1.048 reasonable for Curve pool
8,autoETH,0x896eCc16Ab4AFfF6cE0765A5B924BaECd7Fa455a,osETH/rETH (old),0xe080027Bd47353b5D1639772b4a75E9Ed3658A0d,curve,get_virtual_price()(uint256),WRONG,VP ~1.0025 - SHOULD use same pool as 0x3F55 bu...
9,autoETH,0xe4433D00Cf48BFE0C672d9949F2cd2c008bffC04,pxETH/stETH (main),0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,curve,get_virtual_price()(uint256),OK,VP ~1.01-1.02 reasonable; hardcoded override i...


In [2]:
blocks = build_blocks_to_use(ETH_CHAIN)
surge_pool = "0x6b31a94029fd7840d780191B6D63Fa0D269bd883"
surge_call = Call(
    surge_pool,
    "getRate()(uint256)",
    [("rate", safe_normalize_with_bool_success)],
)
df = get_raw_state_by_blocks([surge_call], blocks, ETH_CHAIN)
df

,rate
timestamp,
2024-09-10 23:59:11+00:00,NaN
2024-09-11 23:57:11+00:00,NaN
2024-09-12 22:57:35+00:00,NaN
2024-09-13 23:58:11+00:00,NaN
2024-09-14 23:59:47+00:00,NaN
...,...
2026-02-22 23:59:47+00:00,1.023298
2026-02-23 23:59:47+00:00,1.023377
2026-02-24 23:59:47+00:00,1.023476


In [3]:
import plotly.express as px

px.line(df, title='wstETH-wETH Surge Fluid VP Over Time')

In [4]:
# vp unsually spike on aug 1 and 2, see where the errors cluster


In [ ]:
def get_chain(autopool_name: str):
    if autopool_name == "baseETH":
        return BASE_CHAIN
    return ETH_CHAIN


CALLABLE_SIGNATURES = {
    "get_virtual_price()(uint256)",
    "getRate()(uint256)",
    "stEthPerToken()(uint256)",
    "exchangePrice()(uint256)",
    "getInvariantDivActualSupply()(uint256)",
}

results = []

for _, row in mapping.iterrows():
    vp_function = row["vp_function"]
    pool_address = row["pool_address"]
    dv = row["destination_vault_address"]
    chain = get_chain(row["autopool"])
    latest_block = chain.get_block_near_top()

    if vp_function not in CALLABLE_SIGNATURES:
        results.append(
            {
                "autopool": row["autopool"],
                "destination_name": row["destination_name"],
                "dv": dv,
                "pool": pool_address,
                "vp_function": vp_function,
                "vp_value": "N/A (special)",
                "block": latest_block,
            }
        )
        continue

    call = Call(
        pool_address,
        vp_function,
        [("vp", safe_normalize_with_bool_success)],
    )
    try:
        state = get_state_by_one_block([call], latest_block, chain)
        vp_value = state["vp"]
    except Exception as e:
        vp_value = f"ERROR: {e}"

    results.append(
        {
            "autopool": row["autopool"],
            "destination_name": row["destination_name"],
            "dv": dv,
            "pool": pool_address,
            "vp_function": vp_function,
            "vp_value": vp_value,
            "block": latest_block,
        }
    )
# aerodrome has 0 fee and base apr?
results_df = pd.DataFrame(results)
results_df

,autopool,destination_name,dv,pool,vp_function,vp_value,block
0,autoETH,autoETH (Idle),0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,N/A - always 1.0,N/A (special),24549872
1,autoETH,Curve.fi Factory Pool: ETHx-ETH,0x1Ea622fa030e4a78F4CC2f305dd3c08DA3F08573,0x59Ab5a5b5d617E478a2479B0cAD80DA7e2831492,get_virtual_price()(uint256),1.056589,24549872
2,autoETH,Curve.fi Factory Pool: ETHx-ETH (old),0xC001f23397dB71B17602Ce7D90a983Edc38DB0d1,0x59Ab5a5b5d617E478a2479B0cAD80DA7e2831492,get_virtual_price()(uint256),1.056589,24549872
3,autoETH,Curve.fi Factory Pool: OETH,0x3772973f8F399D74488D5cF3276C032E0afC8A6f,0x94B17476A93b3262d87B9a326965D1E91f9c13E7,get_virtual_price()(uint256),1.002824,24549872
4,autoETH,OETH/WETH,0x28109255F49f0A02BB31358B3e0b5cb7512EB4eC,0xcc7d5785AD5755B6164e21495E07aDb0Ff11C2A8,get_virtual_price()(uint256),1.001344,24549872
5,autoETH,Instadapp ETH v2,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78,exchangePrice()(uint256),1.202138,24549872
6,autoETH,wstETH (Holding),0xd100c932801390fdeBcE11F26f611D4898b44236,0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0,stEthPerToken()(uint256),1.228044,24549872
7,autoETH,osETH/rETH (new),0x3F55eedDe51504E6Ed0ec30E8289b4Da11EdB7F9,0xe080027Bd47353b5D1639772b4a75E9Ed3658A0d,get_virtual_price()(uint256),1.066379,24549872
8,autoETH,osETH/rETH (old),0x896eCc16Ab4AFfF6cE0765A5B924BaECd7Fa455a,0xe080027Bd47353b5D1639772b4a75E9Ed3658A0d,get_virtual_price()(uint256),1.066379,24549872
9,autoETH,pxETH/stETH (main),0xe4433D00Cf48BFE0C672d9949F2cd2c008bffC04,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,get_virtual_price()(uint256),1.026724,24549872


In [ ]:
suspect_destination_vault_address = '0x5c6aeb9ef0d5BbA4E6691f381003503FD0D45126'
results_df[results_df['dv'] == suspect_destination_vault_address].T

# need the calculator of 0x5c6aeb9ef0d5BbA4E6691f381003503FD0D45126

,25
autopool,autoETH
destination_name,weETH/WETH-ng
dv,0x5c6aeb9ef0d5BbA4E6691f381003503FD0D45126
pool,0xDB74dfDD3BB46bE8Ce6C33dC9D82777BCFc3dEd5
vp_function,get_virtual_price()(uint256)
vp_value,1.067583
block,24530166


In [ ]:
# Aerodrome special case: getK() / totalSupply()
from mainnet_launch.adhoc.fee_and_base_apr_checks.augment_plans import compute_aerodome_vp

aero_dv = "0x945a4f719018edBa445ca67bDa43663C815835Ad"
aero_block = BASE_CHAIN.get_block_near_top()
aero_vp = compute_aerodome_vp(aero_dv, aero_block, BASE_CHAIN)
print(f"Aerodrome weETH/WETH VP (K/totalSupply) at block {aero_block}: {aero_vp:.6e}")

Aerodrome weETH/WETH VP (K/totalSupply) at block 42593907: 2.219218e+20


In [ ]:
# Flag suspicious values
numeric_results = results_df[results_df["vp_value"].apply(lambda x: isinstance(x, (int, float)))].copy()
numeric_results["vp_value"] = numeric_results["vp_value"].astype(float)

suspicious = numeric_results[(numeric_results["vp_value"] < 0.01) | (numeric_results["vp_value"] > 100)]

if len(suspicious) > 0:
    print(f"Found {len(suspicious)} suspicious VP values (< 0.01 or > 100):")
    print(suspicious[["autopool", "destination_name", "vp_function", "vp_value"]].to_string(index=False))
else:
    print("All VP values look reasonable.")

print(f"\nSummary: {len(numeric_results)} callable, {len(results_df) - len(numeric_results)} special/error")

Found 6 suspicious VP values (< 0.01 or > 100):
autopool               destination_name        vp_function     vp_value
 autoETH    Balancer ETHx/wstETH (main) getRate()(uint256) 5.568482e-08
 autoETH     Balancer ETHx/wstETH (old) getRate()(uint256) 5.568482e-08
 autoETH Balancer weETH/rETH StablePool getRate()(uint256) 0.000000e+00
 autoETH        wstETH-rETH-sfrxETH-BPT getRate()(uint256) 2.860760e-06
 baseETH Balancer rETH-WETH Stable Pool getRate()(uint256) 0.000000e+00
 baseETH    Gyroscope ECLP cbETH/wstETH getRate()(uint256) 3.520157e-03

Summary: 30 callable, 3 special/error
